4/25/19
Pandas on ccs and lima score report csv files
Main goal: combine the two csv files, with the rows (indices) being the ZMW hole numbers

Note: axis 1: horizontal; move across all columns
      axis 0: vertical; move across all rows
      
Recall: ccs parameter descriptions
RG Tag: read group id
np: number of passes
rq: predicted read quality
za: avg Z-score of all subreads successfully added


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.close('all')
#if using remote system: files located at: 
# bam_table.csv: /tier2/deweylab/scratch/ipsc_pacbio/demultiplexing/
#                 demultiplexed_full_bams/baseline/absolute_ccs/2_B01/bam_table.csv
#
#score_report_span_0: /tier2/deweylab/scratch/ipsc_pacbio/demultiplexing/
#     demultiplexed_full_bams/baseline/absolute_ccs/2_B01/lima_baseline/
#     ref_span_0/score_report_span_0
#
#
df_ccs = pd.read_csv('/Users/catherineanderson/Documents/Research_Spring_2019/bam_table.csv')
df_ccs = df_ccs.set_index('ZMW hole number')
#dropping RG Tag because it's just an id
df_ccs = df_ccs.drop('RG Tag', axis=1)


df_lima = pd.read_csv('/Users/catherineanderson/Documents/Research_Spring_2019/score_report_span_0.csv')
df_lima = df_lima.set_index('ZMW hole number')



#df_ccs



In [3]:
#df_lima[['IdxsCombined a', 'IdxsCombined b']]
#df_lima


In [4]:
df_combo = pd.merge(df_ccs,df_lima, on="ZMW hole number")

In [5]:
#takes in a GroupBy's object
#returns the number of zmws correctly barcoded
def correct_count (gb):
    count_dict = {}
    for name, group in gb:
        count = 0
  #      print ("name:", name)
  #      print ("group:")
  #      print(group)
        for row_index, row_data in group.iterrows():
 #           print("row_index: ",row_index)
  #          print("row_data: ")
  #          print(row_data)
            first_id = row_data['IdxsCombinedA']
            second_id = row_data['IdxsCombinedB']
  #          print("first id:",first_id)
  #          print("second id:",second_id)
   #     print()
        
            if(first_id == 3):
                if(second_id == 0):
                    count += 1
            elif(first_id == 0):
                if(second_id == 3):
                    count += 1
        count_dict[name] = count
 #   print("count_dict: ",count_dict)
    return count_dict;
        
        


In [6]:
#takes in a GroupBy object 
#returns the (decimal) percent correctly barcoded
def percent_correct(gb):
    correct_count_dict = correct_count(gb)
    group_sizes_dict = gb.size()
    percent_correct_dict = {}
    for name, group in gb:
        percent_correct_dict[name] = correct_count_dict[name]/group_sizes_dict[name]
    return percent_correct_dict
    

Let's do CCS first. Parameters to consider: np (number of passes), rq (predicted read quality), za (avg z-score of zmw's successfully added subreads)

# NP

In [7]:
#need to separate out df_combo based on np; 
#if they have same np, they stay in same mini-dataframe
#print(df_combo.sort_values(by='np'))
#np_grouped = df_combo.groupby('np')
np_percent_correct_dict = percent_correct(df_combo.groupby('np'))
a = 1

In [8]:
np_df = pd.DataFrame.from_dict(np_percent_correct_dict, 'index',columns = ['Fraction correct'])

In [9]:
np_df

Fraction correct
1            0.579576
2            0.811020
3            0.900458
4            0.911166
5            0.914694
6            0.919340
7            0.925019
8            0.915297
9            0.921392
10           0.929353
11           0.927232
12           0.926735
13           0.931421
14           0.936549
15           0.925949
16           0.932639
17           0.934271
18           0.935965
19           0.945271
20           0.951740
21           0.940793
22           0.939427
23           0.946009
24           0.950430
25           0.951169
26           0.951238
27           0.951417
28           0.955629
29           0.943950
30           0.956863
..                ...
196          1.000000
197          1.000000
199          1.000000
200          0.000000
202          1.000000
205          1.000000
210          1.000000
211          0.800000
213          1.000000
221          1.000000
226          1.000000
231          1.000000
233          1.000000
235          1.000000
237          1.000000
239          1.000000
240          1.000000
242          1.000000
248          1.000000
252          1.000000
260          1.000000
263          1.000000
272          1.000000
277          1.000000
278          1.000000
281          1.000000
283          1.000000
287          1.000000
296          1.000000
314          0.000000

[217 rows x 1 columns]

In [ ]:
plt.figure();
np_ax = np_df.plot(kind='barh', use_index = True, figsize=(600,288))
np_ax.set_xlabel('np (number of passes)')
np_ax.set_ylabel('Fraction with Correct Demultiplexing')

Text(0,0.5,'Fraction with Correct Demultiplexing')

<Figure size 432x288 with 0 Axes>